# Homework 3, Group B2

## FINM 37400 - 2024

### UChicago Financial Mathematics

* Mark Hendricks
* hendricks@uchicago.edu

# Group B2

# 1 HBS Case: Fixed-Income Arbitrage in a Financial Crisis (C): Spread and Swap Spread in November 2008

## Simplification of the setup

The date is Nov 4, 2008.

**Treasury bond**
* Suppose the Treasury bond matures exactly 30 years later, on Nov 4, 2038 rather than May 15, 2008. 
* The YTM of this freshly issued treasury is 4.193\% with a semiannual coupon of 4.50\%, same as is given in the case. (So we're just changing the maturity date to simplify things, but keeping the market data.)

**Swap**
* The fixed leg of the swap pays semiannually, with swap rate of 4.2560\%, as given in the case.
* The floating leg of the swap also pays semiannually--not quarterly--such that the payment dates are identical on both legs. Thus, it also resets the floating rate semiannually, not quarterly.
* The floating rate of the swap equals the repo rate used in the trade. Thus, these two rates cancel in the financing of the trade. (No need to consider the TED spread.) 

## Case Clarifications


### Duration Quotes
Bond
* Quote: Val01 of bond is .1746 per bp per \\$1 face value
* Class terminology: Modified dollar duration is .1746 per \\$100 face value

Swap
* Quote: DV01 of swap is 1.7mm per 1 billion notional.
* Class terminology: Modified dollar duration is 100(1.7/1000) per \\$100 face value.

Thus, modified dollar duration for each per 100 face is
* Bond = .1746
* Swap = .1700

### Hedge Ratio

In figuring out the hedge ratio, they set up the hedge per dollar of face value. 

    *so Mills would need to buy face amount \\$0.97 billion**
    
    No, this hedge should be for market value, not face amount given that the case is already using **modified** duration which includes the dirty price.

### Maturity Mismatch

The maturity of the bond is August 2038, whereas the date is Nov 2008. Thus, the bond has less than 30 years to maturity, yet he is entering a 30-year swap. 

For simplicity, we imagine the bond is issued in Nov 2008 and thus has maturity of 30 years at the time of the case.

However, then the case quotes for the Nov price and YTM of the bond no longer are accurate. Use one and adjust the other. Namely, we could...
    * use the Nov 4 **YTM** quoted in the case, and re-adjust the the bond.
    * use the Nov 4 **price** quoted in the case, and re-adjust the YTM.
    
We do the former, keep the quoted YTM, assume time-to-maturity of `30`, and recalculate the Nov 2008 price. (It is close to the quoted price in the case.)

***

##### Defining some functions we may use:

In [1]:
from scipy import optimize as opt

def macDuration(ytm, c, ttm, freq=2):
    # MacaulayDuration based on yield-to-maturity [absolute value], coupon [%], time-to-maturity [years] and frequency [periods per year]
    tp=0
    tt=((ttm*freq)-int(ttm*freq))/freq
    tpp=0
    if tt == 0:
        tp-=c/freq
    while tt < ttm:
        tp+=(c/freq)*(1/((1+(ytm/freq))**(tt*freq)))
        tpp+=tt*(c/freq)*(1/((1+(ytm/freq))**(tt*freq)))
        tt+=1/freq
    tp+=(100+(c/freq))*(1/((1+(ytm/freq))**(ttm*freq)))
    tpp+=ttm*(100+(c/freq))*(1/((1+(ytm/freq))**(ttm*freq)))
    return tpp/tp

def get_price(yield_, maturity, coupon, freq=2):
    # Price based on yield [absolute value], maturity [years], coupon [%], frequency [periods per year] and paid [True/False] (if coupon payment day, whether it was paid)
    tp=0
    tt=((maturity*freq)-int(maturity*freq))/freq
    if tt == 0:
        tp-=coupon/freq
    while tt < maturity:
        tp+=(coupon/freq)/(((1+(yield_/freq))**(freq*tt)))
        tt+=1/freq
    tp+=(100+(coupon/freq))/((1+(yield_/freq))**(maturity*freq))
    return tp

def get_yield(price, maturity, coupon, freq=2):
    # Yield based on price [currency], maturity [years] and coupon [%]
    def bond_price(yield_):
        tp=0
        tt=((maturity*freq)-int(maturity*freq))/freq
        if tt == 0:
            tp-=coupon/freq
        while tt < maturity:
            tp+=(coupon/freq)/(((1+(yield_/freq))**(freq*tt)))
            tt+=1/freq
        tp+=(100+(coupon/freq))/((1+(yield_/freq))**(maturity*freq))
        return price-tp
    return opt.root(bond_price, 0.02).x[0]/365.25

***

## 1.0.

Report the price of the 30-year T-bond in Nov 2008. Given the discussion about `Maturity Mismatch`, we are repricing the bond, so it will not exactly equal `105` as reported in the case.

In [2]:
Bprice = get_price(0.04193, 30, 4.5, 2)
print('USD')
print('The adjusted price of the bond is: ', round(Bprice, 2))

USD
The adjusted price of the bond is:  105.21


## 1.1

List the projected cashflows on May 4, 2009, exactly six months into the trade, on the first coupon and swap date.

* Swap cashflows (before getting net):

  * Fixed-rate payment (payer): -4.2560/2 = -2.128 ($ per $100 face value)
    
  * Floating-rate collection (receiver): repo rate/2 = r2 ($ per $100 face value) 

* Treasury bond cashflow:
        
  * Coupon (collection/receiver): 4.5/2 = 2.25 ($ per $100 face value)

  * Financed through repo (payment): repo rate/2 = r2 ($ per $100 face value)


## 1.2

What is the duration of...
* the T-bond
* the swap

Remember that...
* the swap can be decomposed into a fixed-rate bond and a floating-rate note
* a floating-rate note has duration equal to the time until the next reset. Thus, at initialization, it has duration equal to 0.5 years.

Is the duration for the "paying-fixed" swap positive or negative? Is it bigger or smaller in magnitude than the T-bond?

For this problem, calculate the Macauley duration and the dollar (Macauley) duration.

In [3]:
Bdur=macDuration(0.04193, 4.5, 30, 2)
print('Years')
print('The Macaulay duration of the bond is: ', round(Bdur, 2))

Years
The Macaulay duration of the bond is:  17.08


In [4]:
Sdur=0.5-macDuration(get_yield(100,30,4.256)*365.25, 4.256, 30, 2)
print('Years')
print('The Macaulay duration of the swap is: ', round(Sdur, 2))

Years
The Macaulay duration of the swap is:  -16.71


The duration of the paying-fixed-swap is negative. Similar to being short a fixed-bond (paying) and long a floating rate note (collecting). As a magnitude, the swap's duration is higher than the T-bond, indicating a higher sensitivity to changes in interest rates.

The dollar durations are the following:

In [5]:
print('USD - years')
print('The dollar duration of the swap is: ', round(Sdur*100, 2))
print ('The dollar duration of the bond is: ', round(Bdur*Bprice, 2))

USD - years
The dollar duration of the swap is:  -1671.27
The dollar duration of the bond is:  1797.43


## 1.3

What hedge ratio should be used to balance the notional size of the Treasury bond with the notional size of the swap, such that it is a duration-neutral position?

Specifically, if the trader enters the swap paying fixed on \$500 million notional, how large of a position should they take in the Treasury bond?

In [6]:
nswap=500/100 #number of swaps, adjusting for millions
nbond=-nswap*Sdur/Bdur #number of bonds
print('The number of bonds to hedge the swap is: [millions]', round(nbond, 2)) #Adjusting for millions    
print('The notional of the bonds to hedge the swap is: ', round(nbond*Bprice, 2), 'million USD')


The number of bonds to hedge the swap is: [millions] 4.89
The notional of the bonds to hedge the swap is:  514.65 million USD


## 1.4

Suppose it is May 4, 2009, exactly six months after putting the trade on.

The spread is at -28 bps due to...
* The YTM on a new 30-year bond has risen to 4.36\%
* The swap rate on a new 30-year swap has dropped to 4.08\%

Explain conceptually how this movement impacts the components of the trade.

**Conceptually:**

The trade has lost value. An increase in the YTM on a new 30-year bond means that bonds have decreased in price. Conversely, a drop in the swap rate indicates that for individuals to value the trade's floating and fixed sides equally, the fixed side's price has decreased. 

Both movements are impacting the trade negatively:

* The trade was placed long for the treasury bond at a higher price than today. (Bought too expensive)

* Tre trade was placed paying-fixed for the swap at a higher rate than today. (Bought too expensive)

The trade was betting on the bond's YTM decreasing relative to the swap rate, contrary to what would have occurred in this hypothetical scenario to reach the conditions described for May 4, 2009.

## 1.5

Calculate the value of the position on May 4, 2009, immediately after the first coupon and swap payments and swap reset. 

* Calculate the revised price of the Treasury bond by assuming you can apply the (May 4) 30-year YTM as a discount rate to the 29.5 year bond. (We are just using this for a rough approximation. You know that good pricing would require a discount curve, but let's not get bogged down with that here.)


* Calculate the value of the swap by decomposing it into a fixed-rate bond and a floating-rate bond.
    * The 29.5 year fixed-rate leg is priced using the (May 4) 30-year swap rate as a discount rate.
    * The floating-rate leg is priced at par given that floating-rate notes are par immediately after resets.
    
**Note**

You are being asked to calculate these valuations using the exact formula between price, cashflows, and YTM discount rate. We are not simply approximating with duration, as we already know the position was set up with zero dollar duration.

From the Discussion 1 notebook, we have this formula expressing a bond's price as a function of the coupon, $c$, and the YTM, $y_j$.

$\begin{align}
P_j(t,T,c) = \sum_{i=1}^{n-1}\frac{100\left(\frac{c}{2}\right)}{\left(1+\frac{y_j}{2}\right)^{2(T_i-t)}} + \frac{100\left(1+\frac{c}{2}\right)}{\left(1+\frac{y_j}{2}\right)^{2(T-t)}}
\end{align}
$

In [7]:
Bprice_new=get_price(0.0436, 29.5, 4.5, 2)
Sprice=100 - get_price(0.0408, 29.5, 4.256, 2)
print('On May 4th, 2009. Under assumptions described above: \n')
print('Bonds are now priced at :', round(Bprice_new, 2), 'USD.')
print('The treasury position is valued at: ', round(Bprice_new*nbond, 2), 'million USD. For a loss of :', round((Bprice_new-Bprice)*nbond, 2), 'million USD.\n')
print('Swaps are now priced at :', round(Sprice, 2), 'USD.')
print('The swap position is valued at: ', round(Sprice*nswap, 2), 'million USD. For a loss of :', round((Sprice)*nswap, 2), 'million USD.\n')
print('The value of the position is now: ', round((Bprice_new*nbond)+(Sprice*nswap), 2), 'million USD. For a net value loss of :', round((Bprice_new-Bprice)*nbond + Sprice*nswap, 2), 'million USD.')

On May 4th, 2009. Under assumptions described above: 

Bonds are now priced at : 102.31 USD.
The treasury position is valued at:  500.45 million USD. For a loss of : -14.19 million USD.

Swaps are now priced at : -3.0 USD.
The swap position is valued at:  -15.02 million USD. For a loss of : -15.02 million USD.

The value of the position is now:  485.43 million USD. For a net value loss of : -29.21 million USD.


## 1.6

Accounting for the change in value of the positions, as well as the 6-month cashflows paid on May 4, 
* what is the net profit and loss (pnl) of the position?
* what is the return on the equity capital, considering that there was a 2\% haircut (equity contribution) on the size of the initial treasury bond position.

In [8]:
vchange=((Bprice_new-Bprice)*nbond) + (Sprice*nswap)
cf=(nbond*4.5/2)-(nswap*4.256/2)

In [9]:
#Backup print further below in case you encountered any issue runnng this cell
from IPython.display import display, Markdown

markdown_text = f"""
### Financial Position Analysis as of May 4th, 2009

Given the assumptions described previously, if entering the position for a notional swap value of $500 million USD:

- The value-change in the position is: **{round(vchange, 2)}** million USD.
- The net cash-flow for the position is **{round(cf, 2)}** million USD. (+):
  - Coupon for the bond (collection/receiver): {round(nbond*4.5/2, 2)} million USD.
  - Coupon for the swap (payment/payer): {-nswap*4.256/2} million USD.\\
    _Note: There is a difference in the notional values of the swaps and the value of the bonds. The repo rates (received-swap, paid-bond) are not exactly offsetting each other. However, disregarding this for now._



The net profit and loss (PnL) of the position is **{round(vchange + cf, 2)}** million USD.

The capital invested is (assuming a 2% haircut): {round(nbond*Bprice*0.02, 2)} million USD.\\
The **return on the capital invested** is _**{round(100*(vchange + cf)/(nbond*Bprice*0.02), 2)}%.**_
"""

display(Markdown(markdown_text))



### Financial Position Analysis as of May 4th, 2009

Given the assumptions described previously, if entering the position for a notional swap value of $500 million USD:

- The value-change in the position is: **-29.21** million USD.
- The net cash-flow for the position is **0.37** million USD. (+):
  - Coupon for the bond (collection/receiver): 11.01 million USD.
  - Coupon for the swap (payment/payer): -10.64 million USD.\
    _Note: There is a difference in the notional values of the swaps and the value of the bonds. The repo rates (received-swap, paid-bond) are not exactly offsetting each other. However, disregarding this for now._



The net profit and loss (PnL) of the position is **-28.85** million USD.

The capital invested is (assuming a 2% haircut): 10.29 million USD.\
The **return on the capital invested** is _**-280.25%.**_


***

***

***

*In case for any reason you couldn't see the cell above, 1.6. Same figures*

In [10]:
print('On May 4th, 2009. Under assumptions described above: ')
print('Notional swap value of 500 million USD: \n')


print('The value-change in the position is: ', round(vchange, 2), 'million USD.\n')
print('The cash-flows for the position are: ')
print('Coupon for the bond (collection/receiver): ', round(nbond*4.5/2, 2),'million USD.')
print('Coupon for the swap (payment/payer): ', -nswap*4.256/2,'million USD.')
print('     *** assuming swap collection and treasury repo offset (see note below)')

print('Net cash-flow: ', round(cf, 2),'million USD.\n \n')



print('The net pnl of the position is: ', round(vchange + cf, 2) , 'million USD.\n')

print('The capital invested is (assuming a 2% haircut): ', round(nbond*Bprice*0.02, 2), 'million USD.')
print('The return on the capital invested is: ', round(100*(vchange + cf)/(nbond*Bprice*0.02), 2),'%')

On May 4th, 2009. Under assumptions described above: 
Notional swap value of 500 million USD: 

The value-change in the position is:  -29.21 million USD.

The cash-flows for the position are: 
Coupon for the bond (collection/receiver):  11.01 million USD.
Coupon for the swap (payment/payer):  -10.64 million USD.
     *** assuming swap collection and treasury repo offset (see note below)
Net cash-flow:  0.37 million USD.
 

The net pnl of the position is:  -28.85 million USD.

The capital invested is (assuming a 2% haircut):  10.29 million USD.
The return on the capital invested is:  -280.25 %


There is a difference in the notional values of the swaps and the bonds. The repo rates (received-swap, paid-bond) are not exactly offseting each other. 
However, disregarding this for now.